In [2]:
library(data.table)
library(dtplyr)
library(dplyr, warn.conflicts = FALSE)
library(lubridate)

In [3]:
pangolin <- fread(snakemake@input$pangolin,showProgress = T,nThread = snakemake@threads,verbose = T) %>%
    lazy_dt()

In [4]:
nextclade <- fread(snakemake@input$nextclade,showProgress = T,nThread = snakemake@threads,verbose = T) %>%
    lazy_dt()

In [5]:
meta <- fread(snakemake@input$meta,showProgress = T,nThread = snakemake@threads,verbose = T) %>%
    lazy_dt()

In [7]:
df <- pangolin %>%
    left_join(meta,by=c("taxon"="strain")) %>%
    left_join(nextclade,by=c("taxon"="seqName")) %>%
    as.data.table()

In [9]:
df <- df %>%
    filter(country=="Lithuania") %>%
    as.data.frame()

In [39]:
dfmink <- df %>%
    filter(grepl(pattern = "vison",x = host)  | grepl(pattern = "mink",x = host,ignore.case = T)) %>%
    filter(qc.overallStatus != "bad"  & totalMissing <= 1000) 

nrow(dfmink)


In [44]:
fwrite(x = dfmink,file = snakemake@output$data)

In [43]:
ids <- dfmink %>%
    select(taxon) %>%
    as.data.table()
fwrite(x = ids,file = snakemake@output$ids, col.names = F)